In [2]:
import pandas as pd
import requests

In [9]:
# REGIONALES 
# primero 62
# ultimo 16779
df_regionales = pd.read_json('regionales.json')
df_regionales.to_csv('regionales.csv')

In [10]:
# PROVINCIALES
# primero 17
# ultimo 16856
df_provinciales = pd.read_json('provinciales.json')
df_provinciales.to_csv('provinciales.csv')

In [11]:
# DISTRITALES
# primero 19
# ultimo 16860
df_distritales = pd.read_json('distritales.json')
df_distritales.to_csv('distritales.csv')

In [ ]:
# LISTA DE CANDIDATOS - NACIONAL

for id_solicitud_lista in range(16860):
    url = f'https://apiplataformaelectoral.jne.gob.pe/api/v1/candidato/candidatos-lista-internas/{id_solicitud_lista}'
    resp = requests.get(url=url, verify=False)
    data = resp.json()['data']
    if data != []:
        df = pd.DataFrame(data)
        df = df.drop(['archivoBase64'], axis=1)
        df['idSolicitudLista'] = id_solicitud_lista
        df.to_csv(f"listas/{id_solicitud_lista}.csv")

In [14]:
# LIMPIEZA
df_candidatos = pd.read_csv('candidatos_general.csv')
df_partidos = pd.read_csv('partidos_general.csv')
df_candidatos_final = df_candidatos[df_candidatos.idSolicitudLista.isin(df_partidos.idSolicitudLista)]
df_candidatos_final.to_csv('candidatos_general_limpio.csv')

In [4]:
# ELIMINADOS
df_candidatos_eliminados = df_candidatos[~df_candidatos.idSolicitudLista.isin(df_partidos.idSolicitudLista)]
df_candidatos_eliminados.to_csv('candidatos_general_eliminados.csv')

In [21]:
# MERGE

df1 = pd.read_csv('candidatos_general_limpio.csv')
df2 = pd.read_csv('partidos_general.csv')

df_merged = df1.merge(df2, how='inner', on='idSolicitudLista').drop(columns=['Unnamed: 0.1_x','Unnamed: 0_x','idListaCandidatoInt','idTipoCandidato','idTipoDocumento','txTipoDocumento','idCargoEleccion','txSexo','Unnamed: 0.1_y','Unnamed: 0_y','numCandidatos','idTipoEleccion','numTotalCandidatoHombre','numTotalCandidatoMujer','fgFormula','numNumeroCandidato','fgTieneAccesitario','numTotalAccesitarios','numTotalObligatorios'])

df_merged.to_csv('base_datos_final.csv')

In [6]:
# Numero de partidos: 147
df2 = pd.read_csv('partidos_general.csv')
numero_partidos = df2['txOrganizacionPolitica'].nunique()
numero_partidos

147

In [21]:
# Numero de apellidos unicos: 107330 de 135056 DNIs
df3 = pd.read_csv('base_datos_final.csv')
df3['Apellidos'] = (df3['txNombreCompleto'].str.split().str[-2:].str.join(' '))
personas_unicas = df3['txDocumentoIdentidad'].nunique()
apellidos_unicos = df3['Apellidos'].nunique()
df3.to_csv('base_datos_final.csv')

In [20]:
# DNI duplicados: 369 registros, 184 personas
dni_duplicados = df3[df3['txDocumentoIdentidad'].duplicated(keep=False)]
dni_duplicados.to_csv('dni_duplicados.csv')
personas_unicas = dni_duplicados['txDocumentoIdentidad'].nunique()
personas_unicas
moda = dni_duplicados['txDocumentoIdentidad'].mode()
moda

0    42163374
Name: txDocumentoIdentidad, dtype: int64

In [14]:
# Elecciones regionales: gobernadores y vice gobernadores
df_regionales_gobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_gobernadores = df_regionales_gobernadores[df_regionales_gobernadores['txCargoEleccion']=='GOBERNADOR REGIONAL'].reset_index()
df_regionales_gobernadores.to_csv('gobernadores_regionales.csv')
df_regionales_vicegobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_vicegobernadores = df_regionales_vicegobernadores[df_regionales_vicegobernadores['txCargoEleccion']=='VICEGOBERNADOR REGIONAL'].reset_index()
df_regionales_vicegobernadores.to_csv('vicegobernadores_regionales.csv')

In [ ]:
# Elecciones provinciales: alcaldes
df_provinciales_alcaldes = pd.read_csv('base_datos_final.csv')
df_provinciales_alcaldes = df_provinciales_alcaldes[df_provinciales_alcaldes['txCargoEleccion']=='ALCALDE PROVINCIAL'].reset_index()
df_provinciales_alcaldes.to_csv('alcaldes_provinciales.csv')